In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from CCDPApy.helper import input_path

from CCDPApy import PerfusionCellCulture, PerfusionCellLine, PerfusionExperiment, PerfusionParameters
from CCDPApy import cell_culture_pipeline

pd.set_option('display.max_columns', 200)

In [2]:
param = PerfusionParameters(cell_line_name='CL20',
                            regression_method=['polynomial', 
                                               'rolling_window_polynomial'],
                            rolling_polynomial_degree=3,
                            rolling_polynomial_window=6)
param

Cell Line: CL20
Regression Methods
     Polynomial: True
     Rolling window polynomial True

In [3]:
file = 'perfusion_data.xlsx'
perfusion = cell_culture_pipeline(cell_culture_type='perfusion',
                                  file=file,
                                  parameters=param)

In [ ]:
perfusion.perform_data_process()

In [ ]:
perfusion.save_excel(file_name='perfusion_output.xlsx')

In [ ]:
cell_data = perfusion.get_cell_data().copy()
data = perfusion.get_metabolite_data().copy()

cell_data.keys(), data.keys()

In [4]:
path = input_path('perfusion_data.xlsx')
perfusion = PerfusionCellCulture(parameters=param)
perfusion.load_data(file=path)
cell_culture_data = perfusion.get_all_data()
cell_culture_data.keys()

dict_keys(['measured_data', 'conc', 'feed_conc', 'polynomial_degree_data'])

In [ ]:
perfusion.perform_data_process()

In [ ]:
perfusion.get_metabolite_data()['sp_rate']

In [5]:
cell_line = PerfusionCellLine(cell_line_name='CL20',
                              data=cell_culture_data,
                              parameter=param)
cell_line_data = cell_line.get_all_data()
cell_line_data.keys()

dict_keys(['measured_data', 'conc', 'feed_conc', 'polynomial_degree_data'])

In [ ]:
cell_line.in_process()
cell_line.polynomial()

In [ ]:
cell_line.get_metabolite_data()['sp_rate']

In [6]:
experiment = PerfusionExperiment(cell_line_name='CL20',
                                 cell_line_id='P1',
                                 data=cell_line_data)

In [7]:
experiment.in_process()
experiment.polynomial()
experiment.rolling_window_polynomial()

In [8]:
experiment.sp_rate

,Run Time (day),Run Time (hr),value,unit,method,degree,species,window,ID
0,0.0,0.0,0.496803,(mmol/10^9_cells/hr),polynomial,4,Glucose,NaN,P1
1,0.0,10.0,0.355526,(mmol/10^9_cells/hr),polynomial,4,Glucose,NaN,P1
2,0.0,20.0,0.244385,(mmol/10^9_cells/hr),polynomial,4,Glucose,NaN,P1
3,1.0,25.0,0.243507,(mmol/10^9_cells/hr),polynomial,4,Glucose,NaN,P1
4,1.0,30.0,0.198842,(mmol/10^9_cells/hr),polynomial,4,Glucose,NaN,P1
...,...,...,...,...,...,...,...,...,...
183,7.0,182.5,0.006791,(mmol/10^9_cells/hr),rollingWindowPolynomial,3,Ammonia,6.0,P1
184,7.5,195.0,0.006949,(mmol/10^9_cells/hr),rollingWindowPolynomial,3,Ammonia,6.0,P1
185,8.0,205.0,0.008276,(mmol/10^9_cells/hr),rollingWindowPolynomial,3,Ammonia,6.0,P1
186,8.5,215.0,0.008342,(mmol/10^9_cells/hr),rollingWindowPolynomial,3,Ammonia,6.0,P1


In [ ]:
spc = experiment.get_species()
spc.keys()

In [ ]:
spc['glucose'].

In [ ]:
experiment.sp_rate

In [ ]:
experiment.cumulative_conc

In [ ]:
experiment.sp_rate

In [ ]:
s = experiment.get_species('ammonia')
s.sp_rate_poly

In [ ]:
cell_culture_method = 'perfusion' # 'fed-batch'
polyorder_file='perfusion_polynomial_order.xlsx'
in_file = 'perfusion_temp.xlsx'
kwargs = {'recycling_factor': 0.25, 'concentration_factor': 3,
          'polyreg': True, 'polyorder_file': polyorder_file}

bio_process = bioprocess_pipeline(cell_culture=cell_culture_method,
                                  input_file_name=in_file,
                                  )
bio_process

In [ ]:
cell = bio_process.get_species('cell')
glc = bio_process.get_species('glucose')
glc.get_cumulative

In [ ]:
in_file = 'perfusion_temp.xlsx'
bp = BioProcess(file_name=in_file)
bp.in_process()
bp.polynomial(polyorder_file='perfusion_polynomial_order.xlsx')
bp

In [ ]:
g = bio_process.get_species('cell')
df1 = g.concentration
df2 = g.viability
t = g.get_run_time

# Create scatter plot with Seaborn
fig, ax1 = plt.subplots()
sns.scatterplot(data=df1, x='time', y='value', hue='state', ax=ax1)

# Create a second y-axis
ax2 = ax1.twinx()
sns.lineplot(data=df2, x='time', y='value', ax=ax2, color='r', label='viability')

# Customize the plot
ax1.set_ylabel(f"Concentration {df1['unit'].iat[0]}")
ax2.set_ylabel(f"Viability {df2['unit'].iat[0]}", color='r')
ax1.set_xlabel(f"Time {t['unit'].iat[0]}")
ax1.legend(loc='upper right', bbox_to_anchor=(1.4, 0.9))
ax2.legend(loc='upper right', bbox_to_anchor=(1.4, 1))
plt.show()

In [ ]:
df = bio_process.concentration
df.head()

In [ ]:
df['species'].unique()

In [ ]:
sns.relplot(data=df, x='time', y='value', kind='scatter',
            hue='cellLine', style='runID', 
            col='species',
            )
plt.show()